In [1]:
import requests
import pandas as pd

from datetime import date
from io import BytesIO

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("KMTC")

In [4]:
bl_list 

['KMTCJKT4607234',
 'KMTCJKT4601010',
 'KMTCJKT4605413',
 'KMTCJKT4603863',
 'KMTCJKT4609939',
 'KMTCJKT4603941',
 'KMTCJKT4602791',
 'KMTCJKT4614059',
 'KMTCJKT4613384',
 'KMTCJKT4611583',
 'KMTCJKT4615576',
 'KMTCJKT4615920',
 'KMTCJKT4617862',
 'KMTCJKT4626259']

In [5]:
# array for failed tracking
gagal=[]

In [8]:
hasil_akhir = []
for i, bls in enumerate(tqdm(bl_list)):
    try:
        # hitting the API
        url_ctr = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}?dtKnd=BL&blNo={}".format(
            bls[4:], bls[4:])
        response_ctr = requests.request("GET", url_ctr)

        # getting container data. booking id needed for API URL
        ctr_data = response_ctr.json()
        list_of_container_number = []
        list_of_bookingid = []

        for j, ctr_info in enumerate(ctr_data['cntrList']):
            list_of_container_number.append(ctr_info["cntrNo"])
            list_of_bookingid.append(ctr_info["bkgNo"])

        for c, ctr in enumerate(list_of_container_number):
            url_milestone = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}/detail?bkgNo={}&cntrNo={}&dtKnd=BL&strBkgNo={}".format(
                bls[4:], list_of_bookingid[c], ctr, list_of_bookingid[c])
            
            # hitting the API again for every container for container milestones
            response_milestone = requests.request("GET", url_milestone)

            # aqcuiring milestone datas for every container
            milstone_data = response_milestone.json()
            current_dict = {}
            dict_milestone = {}
            milestones = []
            key_mapping = {
                'cntrNo': 'Container Number',
                'etd': 'ETD',
                'eta': 'ETA',
                # 'GTOOB':'Gate Out Origin',
                # 'GTIOB':'Gate In Origin',
                'LDGOB': 'ATD',
                'DISIB': 'ATA',
                'GTOIB': 'Container Release',
                'GTIIB': 'Container Return'
            }

            current_dict.update({"Liners": "KMTC"})
            current_dict.update({"BL Number": bls})
            current_dict.update(
                {"From": milstone_data['trackingList'][-1]['plcNm'][:milstone_data['trackingList'][-1]['plcNm'].index(",")]})
            current_dict.update(
                {"To": milstone_data['trackingList'][-1]['podPortNm']})

            for key, label in key_mapping.items():
                if key in ctr_data['cntrList'][c]:
                    if key == "etd" or key == "eta":
                        current_dict[label] = datetime.strptime(
                            ctr_data['cntrList'][c][key][:8], "%Y%m%d").strftime("%Y-%m-%d")
                    else:
                        current_dict[label] = ctr_data['cntrList'][c][key]

            for b, milestone in enumerate(milstone_data['trackingList']):
                milestones.append(
                    milestone["cntrStsCd"] + milestone["cntrMvntCd"])
                milestones.append(milestone["mvntDt"])

            for m, milestone in enumerate(milestones):
                if is_date(milestone):
                    case_milestone = {
                        milestones[m-1]: datetime.strptime(milestone, "%Y%m%d").strftime("%Y-%m-%d")}
                    dict_milestone.update(case_milestone)

            for key, label in key_mapping.items():
                if key in dict_milestone:
                    current_dict[label] = dict_milestone[key]

            print(current_dict)
            hasil_akhir.append(current_dict)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)


  0%|          | 0/14 [00:00<?, ?it/s]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4607234', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BEAU4870500', 'ETD': '2023-10-02', 'ETA': '2023-10-06', 'ATD': '2023-10-02', 'ATA': '2023-10-06', 'Container Release': '2023-10-21', 'Container Return': '2023-10-21'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4607234', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BEAU4929357', 'ETD': '2023-10-02', 'ETA': '2023-10-06', 'ATD': '2023-10-02', 'ATA': '2023-10-06', 'Container Release': '2023-10-20', 'Container Return': '2023-10-20'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4607234', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BMOU6239102', 'ETD': '2023-10-02', 'ETA': '2023-10-06', 'ATD': '2023-10-02', 'ATA': '2023-10-06', 'Container Release': '2023-10-23', 'Container Return': '2023-10-24'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4607234', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BMOU6359113', 'ETD': '2023-10-02', 'ETA': '2023-10-06', 'ATD': 

  7%|▋         | 1/14 [00:03<00:40,  3.09s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4607234', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'TXGU5011710', 'ETD': '2023-10-02', 'ETA': '2023-10-06', 'ATD': '2023-10-02', 'ATA': '2023-10-06', 'Container Release': '2023-10-20', 'Container Return': '2023-10-22'}


 14%|█▍        | 2/14 [00:03<00:19,  1.63s/it]

...

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4615920', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'TRHU7001820', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'BMOU6235690', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'BMOU6356602', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'FCIU7148751', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'FCIU7278436', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 

 93%|█████████▎| 13/14 [00:33<00:03,  3.51s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4617862', 'From': 'JAKARTA', 'To': 'SHANGHAI', 'Container Number': 'TXGU5964672', 'ETD': '2023-10-14', 'ETA': '2023-10-26', 'ATD': '2023-10-14'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4626259', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BMOU6241100', 'ETD': '2023-10-19', 'ETA': '2023-10-23', 'ATD': '2023-10-19', 'ATA': '2023-10-23', 'Container Release': '2023-10-24'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4626259', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'BSIU9329308', 'ETD': '2023-10-19', 'ETA': '2023-10-23', 'ATD': '2023-10-19', 'ATA': '2023-10-23'}
{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4626259', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'FCIU7126393', 'ETD': '2023-10-19', 'ETA': '2023-10-23', 'ATD': '2023-10-19', 'ATA': '2023-10-23', 'Container Release': '2023-10-25'}


100%|██████████| 14/14 [00:34<00:00,  2.47s/it]

{'Liners': 'KMTC', 'BL Number': 'KMTCJKT4626259', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'Container Number': 'FFAU1083147', 'ETD': '2023-10-19', 'ETA': '2023-10-23', 'ATD': '2023-10-19', 'ATA': '2023-10-23'}


In [9]:
hasil_akhir

[{'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4607234',
  'From': 'JAKARTA',
  'To': 'HOCHIMINH',
  'Container Number': 'BEAU4870500',
  'ETD': '2023-10-02',
  'ETA': '2023-10-06',
  'ATD': '2023-10-02',
  'ATA': '2023-10-06',
  'Container Release': '2023-10-21',
  'Container Return': '2023-10-21'},
 {'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4607234',
  'From': 'JAKARTA',
  'To': 'HOCHIMINH',
  'Container Number': 'BEAU4929357',
  'ETD': '2023-10-02',
  'ETA': '2023-10-06',
  'ATD': '2023-10-02',
  'ATA': '2023-10-06',
  'Container Release': '2023-10-20',
  'Container Return': '2023-10-20'},
 {'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4607234',
  'From': 'JAKARTA',
  'To': 'HOCHIMINH',
  'Container Number': 'BMOU6239102',
  'ETD': '2023-10-02',
  'ETA': '2023-10-06',
  'ATD': '2023-10-02',
  'ATA': '2023-10-06',
  'Container Release': '2023-10-23',
  'Container Return': '2023-10-24'},
 {'Liners': 'KMTC',
  'BL Number': 'KMTCJKT4607234',
  'From': 'JAKARTA',
  'To': 'HOCHIMINH',
  'Containe

In [7]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(hasil_akhir)

Today's Collection Name ===>  all_tracking_Oct-25-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [8]:
gagal

[]

In [7]:
# exporting to excel file
df = pd.DataFrame(hasil_akhir)
excel_file_path = 'export excel/KMTC.xlsx'
df.to_excel(excel_file_path, index=False)